In [1]:
%cd "~/projects/spartDM"
%load_ext autoreload
%autoreload 2

import os

print(f"Current working dir: {os.getcwd()}")

import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/home/kdang/projects/spartDM
Current working dir: /home/kdang/projects/spartDM


/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/kdang/projects/spartDM/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [16]:
from src.ldae.nets import SemanticEncoder
import torch
import torch.nn as nn
import yaml 

config_path = "configs/starmen_diffae_dim4.yaml"

with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

enc_args = cfg["fit"]["model"]["init_args"]["enc_args"]

encoder = SemanticEncoder(enc_args["backbone_args"], emb_chans=4)

Backbone model loaded successfully.
Frozen first 0 layers out of 161 in the backbone



In [31]:

resnet = encoder.backbone.backbone
for name, layer in resnet.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [65]:
from torchinfo import summary

# Example input: batch_size=32, 3-channel image, 224x224
summary(resnet, input_size=(3, 1, 128, 128), depth=1, col_names=["input_size", "output_size", "num_params", "kernel_size"])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ResNet                                   [3, 1, 128, 128]          [3, 4]                    --                        --
├─Conv2d: 1-1                            [3, 1, 128, 128]          [3, 64, 64, 64]           3,136                     [7, 7]
├─BatchNorm2d: 1-2                       [3, 64, 64, 64]           [3, 64, 64, 64]           128                       --
├─ReLU: 1-3                              [3, 64, 64, 64]           [3, 64, 64, 64]           --                        --
├─MaxPool2d: 1-4                         [3, 64, 64, 64]           [3, 64, 32, 32]           --                        3
├─Sequential: 1-5                        [3, 64, 32, 32]           [3, 256, 32, 32]          215,808                   --
├─Sequential: 1-6                        [3, 256, 32, 32]          [3, 512, 16, 16]          1,219,584                 --
├─Sequentia

In [70]:
import copy

new_resnet = copy.deepcopy(resnet)

In [103]:
from src.ldae.modules.positional_encoding import SinusoidalPosEmb

class SemanticEncoderAgeCond(SemanticEncoder):
    def __init__(self, backbone_args, emb_chans=4):
        super().__init__(backbone_args, emb_chans)

        self.backbone.backbone.fc = nn.Sequential(
            nn.SiLU(),
            nn.Linear(2048, 512),
            nn.SiLU(),
            nn.Linear(512, 128),
            nn.SiLU(),
            nn.Linear(128, 4),
            nn.Sigmoid(),
        )

        self.age_emb = SinusoidalPosEmb(dim=emb_chans)

    def forward(self, x, age):
        age_emb = self.age_emb(age)
        x = self.backbone(x)
        return x + age_emb


new_resnet = SemanticEncoderAgeCond(enc_args["backbone_args"], emb_chans=4)
new_resnet

Backbone model loaded successfully.
Frozen first 0 layers out of 161 in the backbone



SemanticEncoderAgeCond(
  (backbone): BackboneBaseModule(
    (backbone): ResNet(
      (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [102]:
# summary(new_resnet, input_size=(3, 1, 64, 64), depth=3, col_names=["input_size", "output_size", "num_params", "kernel_size"])

dummy = torch.randn(10, 1, 64, 64)
age = torch.rand(10)

test_ysem = new_resnet(x=dummy, age=age)
test_ysem.shape



torch.Size([10, 4])

## Train dataloader

In [129]:
from src.data.datamodules import StarmenDataset
from monai.data import DataLoader
from monai.data.utils import first
import torch
from einops import rearrange

train_ds = StarmenDataset(
    data_dir="data/starmen/output_random_noacc",
    split="train",
    nb_subject=None,
)


train_loader = DataLoader(train_ds, batch_size=3, shuffle=True, num_workers=1)

test_batch = first(iter(train_loader))

In [147]:
ages =test_batch["age"]
ages = rearrange(ages, "b t -> (b t)")

x0 = test_batch["x_origin"]
x0 = rearrange(x0, "b t ... -> (b t) ...")
x0.shape

ages_np = test_batch["age"].cpu().detach().numpy()
ages_np = rearrange(ages_np, "b t -> (b t)")
ages_np

array([-4.26, -3.25, -2.25, -1.24, -0.24,  0.76,  1.77,  2.77,  3.78,
        4.78, -4.81, -3.69, -2.57, -1.46, -0.34,  0.78,  1.9 ,  3.01,
        4.13,  5.25, -0.96,  0.05,  1.05,  2.06,  3.07,  4.07,  5.08,
        6.09,  7.09,  8.1 ], dtype=float32)

In [149]:
test_batch["id"]

tensor([243, 964, 779])

In [136]:
test_ysem = new_resnet(x=x0, age=ages)
test_ysem.shape

torch.Size([30, 4])

In [164]:
ages_all = []
ids_all = []

for i, batch in enumerate(train_loader):
    ## ID 
    ages = batch["age"]
    ages_flat = rearrange(ages, "b t -> (b t)")
    ages_all.append(ages_flat.detach().cpu().numpy())

    ## TIME (age)
    p_ids = batch["id"].cpu().numpy()
    p_ids = np.repeat(p_ids, 10)
    ids_all.append(p_ids)

    # if i == 0:
    #     break


ages_all = np.concatenate(ages_all)
ids_all = np.concatenate(ids_all)

In [166]:
ysem_all = torch.randn(7000, 4)

In [172]:
_, d = ysem_all.shape
feature_cols = [f'dim_{i}' for i in range(d)]
ysem_df = pd.DataFrame(ysem_all, columns=feature_cols)
ysem_df.index = pd.MultiIndex.from_arrays([ids_all.astype("str"), ages_all], names=["ID", "TIME"])

ysem_df.index.is_unique
ysem_df.loc[["1"]]

dim_0     dim_1     dim_2     dim_3
ID TIME                                         
1  -6.55  0.102416 -0.456615  0.327368 -0.442899
   -5.56 -1.116357  1.146028 -0.518082 -0.573103
   -4.57 -1.323636  0.469699 -0.741832  2.313334
   -3.58 -0.774971 -0.715238  0.396925 -0.743602
   -2.60 -0.235038  1.227362  0.123059 -1.128107
   -1.61 -1.556063 -1.452566 -0.204208  0.232274
   -0.62  0.317203 -0.922622 -1.723516  0.080623
    0.37 -0.151098  1.349612  1.164649 -0.884852
    1.36  0.809014  0.075111 -1.034793  0.942618
    2.35 -1.249408  0.352551 -0.225258  1.111448